In [ ]:
# # Run this and then restart the kernel at the start of each session to install
# # 'teotil3' in development mode
# !pip install -e /home/jovyan/projects/teotil3/

In [ ]:
# !pip install fpdf

In [1]:
import calendar

import matplotlib.pyplot as plt
import nivapy3 as nivapy
import pandas as pd
import teotil3 as teo

plt.style.use("ggplot")

In [2]:
eng = nivapy.da.connect_postgis()

Connection successful.


# TEOTIL3 Trondheimsfjorden

# Part 8c: Explore results for all catchments

## 1. All catchments > 100 km2

In [3]:
nve_data_year = 2024
st_yr, end_yr = 2020, 2023
result_csv = f"/home/jovyan/shared/common/teotil3/trondheimsfjorden_scenarios/trondheimsfjorden_results_nve{nve_data_year}_2013-{end_yr}.csv"
pars = ["totn", "totp"]
ges_dict = {"totn": 775, "totp": 29}
area_thresh_km2 = 100

In [4]:
# Read saved data
mod_df = pd.read_csv(result_csv)

# Clip to period of interest
mod_df = mod_df.query("(year >= @st_yr) and (year <= @end_yr)")

mod_df.head()

,scenario,year,regine,regine_down,accum_agriculture-background_din_kg,accum_agriculture-background_ss_kg,accum_agriculture-background_tdp_kg,accum_agriculture-background_toc_kg,accum_agriculture-background_ton_kg,accum_agriculture-background_totn_kg,...,local_urban_totp_kg,local_urban_tpp_kg,local_wood_din_kg,local_wood_ss_kg,local_wood_tdp_kg,local_wood_toc_kg,local_wood_ton_kg,local_wood_totn_kg,local_wood_totp_kg,local_wood_tpp_kg
9387,Baseline,2020,120.111,120.,461.163770,9638.603285,5.622947,12392.295270,119.241995,580.405765,...,49.0,19.6,207.1,6148.3,9.8,95386.6,1761.8,1968.9,56.3,46.5
9388,Baseline,2020,120.1120,120.,162.799950,3831.254932,2.206947,4439.847272,45.945729,208.745679,...,3.0,1.2,44.1,1295.5,2.1,20126.9,378.2,422.3,12.1,10.0
9389,Baseline,2020,120.112Z,120.,104.974967,181.844889,1.061741,2535.246117,29.312484,134.287451,...,0.0,0.0,100.9,2719.9,4.8,45885.3,859.9,960.8,27.5,22.7
9390,Baseline,2020,120.11Z,120.,70.650149,38.413259,1.816932,2977.699354,23.685912,94.336061,...,0.0,0.0,266.5,6800.2,12.6,120666.3,2258.0,2524.5,72.4,59.8
9391,Baseline,2020,120.12,120.,171.539142,3885.054499,2.185559,4339.969759,45.187266,216.726408,...,0.0,0.0,149.3,4250.9,7.1,66961.3,1276.4,1425.7,41.5,34.4


In [5]:
def days_in_year(year):
    return 366 if calendar.isleap(year) else 365


# Get cols of interest
id_cols = ["scenario", "year", "regine", "regine_down"]
cols = [col for col in mod_df.columns if col.startswith("accum_")]
df = mod_df[id_cols + cols].copy()
df.columns = [
    col.replace("accum_", "") if col.startswith("accum_") else col for col in df.columns
]

# Only catchments greater than min size
df = df.query("upstr_area_km2 > @area_thresh_km2")

# Annual flow volume
df["days_in_year"] = df["year"].apply(days_in_year)
df["flow_vol_l"] = df["q_m3/s"] * 60 * 60 * 24 * df["days_in_year"] * 1000

# Totals for each par
for par in pars:
    par_cols = [col for col in df.columns if f"_{par}_" in col]
    df[f"total_{par}_kg"] = df[par_cols].sum(axis="columns")

# Sum over time period
df = df.groupby(["scenario", "regine", "regine_down"]).sum().reset_index()
del df["year"], df["days_in_year"]

# Mean concs over period
for par in pars:
    df[f"conc_{par}_ugpl"] = df[f"total_{par}_kg"] * 1e9 / df["flow_vol_l"]

cols = [col for col in df.columns if any(f"_{par}_" in col for par in pars)]
df = df[["scenario", "regine", "regine_down", "upstr_area_km2"] + cols]

In [ ]:
# All regines with upstream area greater than size threshold that do not meet GES for 'baseline'.
df.query(
    f"(scenario == 'Baseline') and "
    f"((conc_totn_ugpl > {ges_dict['totn']}) or (conc_totp_ugpl > {ges_dict['totp']}))"
).to_excel("test1.xlsx", index=False)

## 2. Local "hot spots"

In [ ]:
# Get cols of interest
id_cols = ["scenario", "year", "regine", "regine_down"]
cols = [col for col in mod_df.columns if col.startswith("local_")]
df = mod_df[id_cols + cols].copy()
df.columns = [
    col.replace("local_", "") if col.startswith("local_") else col for col in df.columns
]

# Annual flow volume
df["days_in_year"] = df["year"].apply(days_in_year)
df["flow_vol_l"] = df["q_cat_m3/s"] * 60 * 60 * 24 * df["days_in_year"] * 1000

# Totals for each par
for par in pars:
    par_cols = [col for col in df.columns if f"_{par}_" in col]
    df[f"total_{par}_kg"] = df[par_cols].sum(axis="columns")

# Sum over time period
df = df.groupby(["scenario", "regine", "regine_down"]).sum().reset_index()
del df["year"], df["days_in_year"]

# Mean concs over period
for par in pars:
    df[f"conc_{par}_ugpl"] = df[f"total_{par}_kg"] * 1e9 / df["flow_vol_l"]

df["a_cat_land_km2"] = df["a_cat_land_km2"] / (end_yr - st_yr + 1)
cols = [col for col in df.columns if any(f"_{par}_" in col for par in pars)]
df = df[["scenario", "regine", "regine_down", "a_cat_land_km2"] + cols]

In [ ]:
# All regines with local inputs greater than threshold for GES for 'baseline'.
df2 = df.query(
    f"(scenario == 'Baseline') and "
    f"((conc_totn_ugpl > {ges_dict['totn']}) or (conc_totp_ugpl > {ges_dict['totp']}))"
).copy()
df2.to_excel("test2.xlsx", index=False)

In [ ]:
df2["pct_totn_lrgww"] = 100 * df2["large-wastewater_totn_kg"] / df2["total_totn_kg"]
df2 = df2.query("pct_totn_lrgww < 50")

In [ ]:
reg_list = df2["regine"].tolist()
reg_gdf = (
    teo.io.get_regine_geodataframe(eng, 2023)
    .query("regine in @reg_list")
    .to_crs("epsg:4326")
)
reg_gdf = reg_gdf.merge(df2, how="left", on="regine")

In [ ]:
import folium

# Create a map centered around the mean coordinates of the geodataframe
m = folium.Map(
    location=[reg_gdf.geometry.centroid.y.mean(), reg_gdf.geometry.centroid.x.mean()],
    zoom_start=5,
)

# Add the choropleth layer
folium.Choropleth(
    geo_data=reg_gdf,
    name="choropleth",
    data=reg_gdf,
    columns=["regine", "conc_totn_ugpl"],
    key_on="feature.properties.regine",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Nitrogen Concentration (ug/L)",
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

m